# Introduction

<div class="alert alert-block alert-success">
    Books are valuble materials for humans. Throughout history, the books industry has evolved from hand written books, to mechanical typing machines, and now to the age of electronic, and audio books. Amazon, Google and many other companies are investing in this field. As the books industry evolved, the number of books has increased dramatically giving the readers very wide variaty of books materials and many options. However, this also put the readers in so much confusion.

# Problem Statment

<div class="alert alert-block alert-success">
    To make it easier for readers it is very important to classify the books accuratly depending on thier content. Each single book must be labeled according to the title and the description of the book. This process of assigning books to different generes when done manually, can be very tidues, time consuming and costy. <br>
    
    
   Machine Learning with the aid of Natural Language Processing (NLP), can help automating this process and thus, saving the companies both time and money.
   
   In this project our aim is to design a classifier that can assign genere to each book, given their title and description. NLP would be used to process the title and the description of each book before using a Naive Bayes classifier to classify the books.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem import WordNetLemmatizer, StemmerI, PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stemming.porter2 import stem

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('google_books_1299.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
df.head()

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192,none,9781612626864,English,"Jul 31, 2014"
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9780758272799,English,"Jul 1, 2007"
2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368,"Games &amp, Activities , Video &amp, Electronic",9781506713816,English,"Nov 5, 2019"
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320,none,9781617734076,English,"Mar 1, 2009"
4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544,"Fiction , Fantasy , Dark Fantasy",9780007287758,English,"Jan 8, 2009"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 13 columns):
title             1299 non-null object
author            1299 non-null object
rating            1224 non-null float64
voters            1224 non-null object
price             1299 non-null float64
currency          1299 non-null object
description       1296 non-null object
publisher         1299 non-null object
page_count        1299 non-null int64
generes           1299 non-null object
ISBN              1299 non-null object
language          1299 non-null object
published_date    1299 non-null object
dtypes: float64(2), int64(1), object(10)
memory usage: 132.0+ KB


# Data Descreption

|Feature|Type|Description|
|---|---|---|
|title|object|The title of the book|,
|author|object|The author of the book|,
|rating|float|The rating of the book by the customers|,
|voters|int|The number of customers who voted|,
|price|float|The book price on Google store|,
|currency|object|The currency in which the price is reported|,
|description|object|A breif description of the book given in the store|,
|publisher|object|The publisher of the book|,
|page_count|int|The number of pages of the book|,
|generes|object|The generes under which the book is catogrized|,
|ISBN|object|the unique identification of each book|
|language|object|The language in which the book is written|
|published_date|object|The date at which the book was published|

In [6]:
df.describe()

,rating,price,page_count
count,1224.000000,1299.000000,1.299000e+03
mean,4.424755,45.882733,1.505974e+10
std,0.392489,53.588444,3.836540e+11
min,2.500000,0.000000,7.000000e+00
25%,4.200000,22.470000,1.940000e+02
50%,4.500000,38.330000,3.040000e+02
75%,4.700000,59.030000,4.160000e+02
max,5.000000,886.080000,9.781302e+12


**We have some duplicates so we will drop them.**

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.shape

(948, 13)

**Our target is to predict two generes for each book, so we will create two new columns for each row**

In [9]:
df['first_genere'] = df.generes.str.split(',',expand=True)[0]
df['second_genere'] = df.generes.str.split(',',expand=True)[1]

**We will create a new column that incudes both the title and the description of each book**

In [10]:
df["text"] = df["title"].map(str) + ', ' + df["description"].str.strip('\n\t')

In [11]:
df.head(2)

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date,first_genere,second_genere,text
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192,none,9781612626864,English,"Jul 31, 2014",none,None,"Attack on Titan: Volume 13, NO SAFE PLACE LEFT..."
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9780758272799,English,"Jul 1, 2007",Fiction,Mystery &amp,Antiques Roadkill: A Trash 'n' Treasures Myste...


**We can see that we have some books with missing geners. Thus we will isolate these books to predict their generes after devoloping our model**

In [81]:
pred_df = df[df.generes=='none']

In [82]:
pred_df.isnull().sum()

title               0
author              0
rating             13
voters             13
price               0
currency            0
description         3
publisher           0
page_count          0
generes             0
ISBN                0
language            0
published_date      0
first_genere        0
second_genere     227
text                3
dtype: int64

In [83]:
pred_df.dropna(subset=['text'],inplace=True)
X_pred = pred_df.text
pred_df.shape,X_pred.shape

((224, 16), (224,))

**The observations that have generes will be saved in another dataframe (train_test_df)**

In [17]:
train_test_df = df[df.generes!='none']

In [18]:
train_test_df.shape

(721, 16)

**Now that we have a dataframe with the desired features, we can take X and y variable. Remeber that we need to predict two generes for each book, and therefore, we will have two y's (y1 and y2).**

In [19]:
X = train_test_df.text
y = train_test_df[['first_genere']]

In [20]:
X[:2]

1    Antiques Roadkill: A Trash 'n' Treasures Myste...
2    The Art of Super Mario Odyssey, Take a globetr...
Name: text, dtype: object

In [21]:
y.head(2)

,first_genere
1,Fiction
2,Games &amp


# NLP Preprocessing

### RegularExpression To Remove the Punctiuation. Inadition to stemming.

In [22]:
import regex as re

In [23]:
def text_preprocessor(text):
    text = [re.sub('[^A-Za-z]+n?',' ',x).lower() for x in text]
    books = []
    for book in text:
        book_token = book.split()
        book = " ".join([stem(x.replace(' ','')) for x in book_token])
        books.append(book)
    text = books
    
    return text

In [24]:
X[1] # text before Cleaning

'Antiques Roadkill: A Trash \'n\' Treasures Mystery, Determined to make a new start in her quaint hometown on the banks of the Mississippi, Brandy Borne never dreams she\'ll become the prime suspect in a murder case. . . \n\nMoving back in with her eccentric, larger-than-life mother, Brandy Borne finds small-town Serenity anything but serene. It seems an unscrupulous antiques dealer has swindled Vivian out of the family\'s heirlooms. But when he is found run over in a country lane, Brandy becomes Murder Suspect Number One--with her mother coming in a very close second. . . \n\nThe list of other suspects is impressive--the victim\'s business seems to have been based on bilking seniors out of their possessions. And when the Borne "girls" uncover a few very unsavory Serenity secrets, they become targets for a murderer whose favorite hobby seems to be collecting victims. \n\nDon\'t miss Brandy Borne\'s tips on antiques! \n\n"Cozy mystery fans will love this down-to-earth heroine with the w

In [25]:
X = text_preprocessor(X)

In [26]:
X[0] # Text after Cleaning

'antiqu roadkil a trash treasur mysteri determin to make a ew start in her quaint hometown on the bank of the mississippi brandi born ever dream she ll becom the prime suspect in a murder case move back in with her eccentr larger than life mother brandi born find small town seren anyth but seren it seem an unscrupul antiqu dealer has swindl vivian out of the famili s heirloom but when he is found run over in a countri lane brandi becom murder suspect number one with her mother come in a veri close second the list of other suspect is impress the victim s busi seem to have been base on bilk senior out of their possess and when the born girl uncov a few veri unsavori seren secret they becom target for a murder whose favorit hobbi seem to be collect victim don t miss brandi born s tip on antiqu cozi mysteri fan will love this down to earth heroin with the wri sens of humor and a big heart nanci pickard with it small town set and quirki charact antiqu roadkil is fun from start to finish div

In [27]:
vectorizer = CountVectorizer(analyzer='word',
                             max_df=0.9,
                             max_features=5000, 
                             min_df=3,
                             stop_words= stopwords.words('english'),
                                )

In [28]:
vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.9, max_features=5000, min_df=3,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [29]:
X_vec = vectorizer.transform(X)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_vec.toarray() , y , test_size=0.3333 , random_state=42)

In [31]:
# y_train1 = y_train.first_genere
# y_train2 = y_train.second_genere

# y_test1 = y_test.first_genere
# y_test2 = y_test.second_genere

In [32]:
# X_train = vectorizer.transform(X_train)

In [33]:
sparse_train = pd.DataFrame(X_train, columns = vectorizer.get_feature_names())
sparse_train.head(2)

,abaddon,abandon,abc,abdic,abduct,abductor,abel,abercrombi,abil,abl,...,youth,ysann,zaha,zahn,zamyatin,zap,zealand,zeinab,zero,zone
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [34]:
# X_test =  vectorizer.transform(X_test)

In [35]:
sparse_test = pd.DataFrame(X_test, columns = vectorizer.get_feature_names())
sparse_test.head(2)

,abaddon,abandon,abc,abdic,abduct,abductor,abel,abercrombi,abil,abl,...,youth,ysann,zaha,zahn,zamyatin,zap,zealand,zeinab,zero,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### MultinomiaNB using CountVectorizer

In [36]:
model_nb = MultinomialNB()
# model_nb2 = MultinomialNB()

In [37]:
model_nb.fit(X_train, y_train)
# model_nb2.fit(X_train, y_train2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
model_nb.score(X_train, y_train)

0.85625

In [39]:
model_nb.score(X_test, y_test)

0.7427385892116183

In [40]:
# model_nb2.score(X_train, y_train2)

In [41]:
# model_nb2.score(X_test, y_test2)

### RandomForest using CountVectorizer

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rfc = RandomForestClassifier()
# rfc2 = RandomForestClassifier()

In [44]:
rfc.fit(X_train,y_train)
# rfc2.fit(X_train,y_train2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [45]:
X_train.shape

(480, 4615)

In [46]:
rfc.score(X_train,y_train)

0.8708333333333333

In [47]:
rfc.score(X_test, y_test)

0.7095435684647303

In [48]:
# rfc2.score(X_train,y_train2)     

In [49]:
# rfc2.score(X_test, y_test2)

### MultinomiaNB using TF-IDF

In [50]:
from sklearn.feature_extraction.text import TfidfTransformer

In [55]:
model_nb_tfidf = MultinomialNB()

In [56]:
tfidf_trans = TfidfTransformer()
tfidf_trans.fit(sparse_train)

# tfidf_trans2 = TfidfTransformer()
# tfidf_trans2.fit(sparse_train)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [57]:
train_tfidf = tfidf_trans.transform(sparse_train)

In [58]:
test_tfidf = tfidf_trans.transform(sparse_test)

In [59]:
model_nb_tfidf.fit(train_tfidf, y_train)
# model_nb2.fit(train_tfidf, y_train2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [60]:
model_nb_tfidf.score(train_tfidf, y_train)

0.6916666666666667

In [61]:
# model_nb2.score(train_tfidf, y_train2)

In [62]:
model_nb_tfidf.score(test_tfidf, y_test)

0.6763485477178424

In [ ]:
# model_nb2.score(test_tfidf, y_test2)

### RandomForest using TF-IDF

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
rfc_tfidf = RandomForestClassifier()
# rfc2 = RandomForestClassifier()

In [65]:
rfc_tfidf.fit(train_tfidf, y_train)
# rfc2.fit(X_train,y_train2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [66]:
rfc_tfidf.score(train_tfidf, y_train)

0.8708333333333333

In [67]:
rfc_tfidf.score(X_test, y_test)

0.6431535269709544

### Building a pipline with the best model and performing  GridSearchCV on the CountVectorizer

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [69]:
pipe = Pipeline([('cvec', vectorizer), ('mnb', model_nb)])

In [70]:
pipe_params = {
    'cvec__max_features':[5000, 6000, 7000, 8000,None],
    'cvec__min_df':[1,2, 3],
    'cvec__max_df':[0.9, 0.95,1.0],
    'cvec__ngram_range': [(1, 1), (1, 2)],
    'cvec__stop_words' : [stopwords.words("english"), None]
                }
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [71]:
gs = GridSearchCV(pipe, pipe_params, cv=3, verbose=1, n_jobs=-1)

In [72]:
gs.fit(X, y)

Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('cvec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.9,
                                                        max_features=5000,
                                                        min_df=3,
                                                        ngram_range=(1, 1),
                                            

In [73]:
gs.best_estimator_

Pipeline(memory=None,
         steps=[('cvec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.9,
                                 max_features=5000, min_df=3,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
   

In [74]:
gs.best_score_

0.7503467406380028

<div class="alert alert-block alert-success">

The best estimator is the same as the one used.

### predictting the generes for the books with 'none' in the column of genere.

In [84]:
X_pred = text_preprocessor(X_pred)

In [85]:
X_pred_sparse = vectorizer.transform(X_pred)

In [86]:
pred_df.generes = model_nb.predict(X_pred_sparse.toarray())

In [89]:
pred_df.head(2)

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date,first_genere,second_genere,text
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192,Comics &amp,9781612626864,English,"Jul 31, 2014",none,None,"Attack on Titan: Volume 13, NO SAFE PLACE LEFT..."
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320,Fiction,9781617734076,English,"Mar 1, 2009",none,None,Getting Away Is Deadly: An Ellie Avery Mystery...


# Conclusion & Recommendation

<div class="alert alert-block alert-success">

- In this project a classification model was developed using NLP analysis and Naieve Bayes Models.
- The best Accuracy found was 0.7427385892116183 for the test and 0.8708333333333333 for train data. 
- The score can be enhanced with more data.
- We recommend that more books be scraped from google books store to achieve higher accuracy for the model.
- It is also recommewnded to build a recommender system and use the NLP analysis to help enhancing the recommendations accuracy.